# Final Project : Use ensemble methods and compare the results

**Medical Insurance Charges**

The goal of this project is to predict an individual’s medical insurance charges based on various factors.

Premiums for health insurance can vary based on lifestyle choices, health conditions, and other factors.

I’d like to determine which factors are most influential towards insurances costs and have a model that will predict charges for individuals based on their attributes.

## Load Data ##
We'll use the dataset at https://www.kaggle.com/datasets/mirichoi0218/insurance

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

dataset = "mirichoi0218/insurance"
file_name = "insurance.csv"

df = kagglehub.dataset_load(KaggleDatasetAdapter.PANDAS, dataset, file_name)
raw = df.copy()

## Visualize Data ##

In [ ]:
display(
    raw.shape,
    raw.head(10),
    raw.tail(10),
)

In [ ]:
display(raw.isnull().sum())
prev_shape = raw.shape
raw = raw.dropna()
shape = raw.shape
if prev_shape == shape:
    print("[INFO] shapes stayed the same after dropna() call")
else:
    print("[WARN] shape changed")

In [ ]:
raw = raw.drop_duplicates()

display(
    raw.info(),
    raw.describe().T,
    raw.duplicated().sum(),
    raw.shape,
)

In [ ]:
# see aggregated counts for cols
cols = ["children", "region", "sex", "smoker"]
for col in cols:
    display(raw[col].value_counts())

In [ ]:
import matplotlib.pyplot as plt

raw.hist(["age", "bmi", "charges"])
plt.show()

In [ ]:
import numpy as np
import seaborn as sns

raw["charges"] = np.log(raw["charges"] + 1)
sns.histplot(raw["charges"])
plt.title("charges log")
plt.show()

In [ ]:
import pandas as pd

raw = pd.get_dummies(
    raw, columns=["sex", "smoker", "region"], drop_first=True, dtype=int
)

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(raw.corr(), annot=True, cmap="coolwarm")
plt.show()

**Correlations of charges with smoker_yes and age**

In [ ]:
raw_corr = raw.copy()
corr_matrix = raw_corr[["age", "smoker_yes", "charges"]].corr()

plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation with Charges", fontsize=14)
plt.show()

In [ ]:
sns.regplot(
    data=raw,
    x="age",
    y="charges",
    scatter_kws={"alpha": 0.4},
    line_kws={"color": "red"},
)
plt.title("Linear Relationship: Age vs Charges")
plt.xlabel("Age")
plt.ylabel("Charges")
plt.show()

In [ ]:
sns.boxplot(data=raw, x="smoker_yes", y="charges", hue="smoker_yes", palette="Set2")
plt.title("Charges by Smoking Status")
plt.show()

## Preprocess Data ##

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    BaggingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
    StackingRegressor,
    VotingRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor


numeric_features = ["age", "bmi", "children"]
categorical_features = ["sex", "smoker", "region"]

df = df.drop_duplicates()


X = df[numeric_features + categorical_features]
X = df.drop(columns=["charges"])
y = df["charges"]
y_log = np.log1p(df["charges"])

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()
X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

## Ensemble Methods/Models ##


- BaggingRegressor
- GradientBoostingRegressor
- RandomForestRegressor
- StackingRegressor
- VotingRegressor

We also have LinearRegression and KNNRegressor.


### Tune ###

In [ ]:
from sklearn.model_selection import GridSearchCV

base_models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Bagging": BaggingRegressor(estimator=DecisionTreeRegressor(), random_state=42),
    "Stacking": StackingRegressor(
        estimators=[
            ("rf", RandomForestRegressor(random_state=42)),
            ("gb", GradientBoostingRegressor(random_state=42)),
        ],
        final_estimator=LinearRegression(),
    ),
    "Voting": VotingRegressor(
        [
            ("lr", LinearRegression()),
            ("rf", RandomForestRegressor(random_state=42)),
            ("gb", GradientBoostingRegressor(random_state=42)),
        ]
    ),
}


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        (
            "cat",
            OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore"),
            categorical_features,
        ),
    ]
)


param_grids = {
    "Gradient Boosting": {
        "regressor__n_estimators": [100, 200],
        "regressor__max_depth": [3, 5],
        "regressor__learning_rate": [0.05, 0.1],
    },
    "Random Forest": {
        "regressor__n_estimators": [100, 200],
        "regressor__max_depth": [None, 10],
    },
    "Decision Tree": {
        "regressor__max_depth": [None, 5, 10],
        "regressor__min_samples_split": [2, 5],
    },
    "Bagging": {
        "regressor__n_estimators": [10, 50],
        "regressor__max_samples": [0.8, 1.0],
    },
}

tuned_models = ["Gradient Boosting", "Random Forest", "Decision Tree", "Bagging"]

best_models = {}

for name in tuned_models:
    model = base_models[name]
    pipe = Pipeline([("preprocessor", preprocessor), ("regressor", model)])

    grid = GridSearchCV(pipe, param_grids[name], cv=5, scoring="r2", n_jobs=-1)
    grid.fit(X_train, y_train)

    best_models[name] = grid.best_estimator_

    print(f"\n{name} - Best R²: {grid.best_score_:.4f}")
    print(f"Best Params: {grid.best_params_}")

### Evaluate Models ###

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(
        max_depth=5, min_samples_split=2, random_state=42
    ),
    "Random Forest": RandomForestRegressor(
        max_depth=10, n_estimators=200, random_state=42
    ),
    "Gradient Boosting": GradientBoostingRegressor(
        learning_rate=0.05, max_depth=3, n_estimators=100, random_state=42
    ),
    "Bagging": BaggingRegressor(
        max_samples=0.8,
        n_estimators=50,
        estimator=DecisionTreeRegressor(),
        random_state=42,
    ),
    "Stacking": StackingRegressor(
        estimators=[
            (
                "rf",
                RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42),
            ),
            (
                "gb",
                GradientBoostingRegressor(
                    learning_rate=0.05, max_depth=3, n_estimators=100, random_state=42
                ),
            ),
        ],
        final_estimator=LinearRegression(),
    ),
    "Voting": VotingRegressor(
        [
            (
                "rf",
                RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42),
            ),
            (
                "gb",
                GradientBoostingRegressor(
                    learning_rate=0.05, max_depth=3, n_estimators=100, random_state=42
                ),
            ),
        ]
    ),
}

best_r2 = -float("inf")
best_rmse = float("inf")
best_model_r2 = None
best_model_rmse = None
feature_importances = {}

for name, model in models.items():
    pipe = Pipeline([("preprocessor", preprocessor), ("regressor", model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    preds_original_scale = np.exp(preds)
    y_test_original_scale = np.exp(y_test)

    r2 = r2_score(y_test_original_scale, preds_original_scale)
    rmse = mean_squared_error(y_test_original_scale, preds_original_scale) ** 0.5
    mae = mean_absolute_error(y_test_original_scale, preds_original_scale)

    if r2 > best_r2:
        best_r2 = r2
        best_model_r2 = name

    if rmse < best_rmse:
        best_rmse = rmse
        best_model_rmse = name

    print(f"\n{name} - Predictions:")
    print(f"r2: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print("\nPredictions vs Actuals:")

    comparison = pd.DataFrame(
        {"Actual": y_test_original_scale, "Predicted": preds_original_scale}
    )
    print(comparison.head())

    if hasattr(model, "feature_importances_"):
        feature_importances[name] = model.feature_importances_

print(f"\nBest Model by r2: {best_model_r2} with r2 = {best_r2:.4f}")
print(f"\nBest Model by RMSE: {best_model_rmse} with RMSE = {best_rmse:.4f}")

In [ ]:
# cross-validation
# highest R^2 and lowest RMSE

model_r2_scores = {}
model_rmse_scores = {}

for name, model in models.items():
    pipe = Pipeline([("preprocessor", preprocessor), ("regressor", model)])

    r2_scores = cross_val_score(pipe, X, y, cv=5, scoring="r2")
    rmse_scores = cross_val_score(
        pipe, X, y, cv=5, scoring="neg_root_mean_squared_error"
    )

    model_r2_scores[name] = r2_scores.mean()
    model_rmse_scores[name] = -rmse_scores.mean()

    print(f"\n{name} - Cross-Validation Results:")
    print(f"Mean R²: {model_r2_scores[name]:.4f}")
    print(f"Mean RMSE: {model_rmse_scores[name]:.4f}")

In [ ]:
# highest R^2 and lowest RMSE

r2_scores = []
rmse_scores = []
model_names = []

for name, model in models.items():
    pipe = Pipeline([("preprocessor", preprocessor), ("regressor", model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    r2 = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    r2_scores.append(r2)
    rmse_scores.append(rmse)
    model_names.append(name)

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].barh(model_names, r2_scores, color="skyblue")
ax[0].set_title("r2 Scores")
ax[0].set_xlabel("r2")
ax[0].set_ylabel("Model")

ax[1].barh(model_names, rmse_scores, color="salmon")
ax[1].set_title("RMSE Scores")
ax[1].set_xlabel("RMSE")
ax[1].set_ylabel("Model")

plt.tight_layout()
plt.show()

### Feature Importance ###

In [ ]:
for name, importances in feature_importances.items():
    if importances is not None:
        if "Stacking" not in name and "Voting" not in name:
            encoded_columns = preprocessor.transformers_[1][1].get_feature_names_out(
                categorical_features
            )
            all_columns = numeric_features + list(encoded_columns)

            plt.figure(figsize=(10, 6))
            plt.barh(all_columns, importances)
            plt.xlabel("Importance")
            plt.title(f"Feature Importances for {name}")
            plt.show()

# Observations

### Linear Regression
- **R²**: 0.7181  
- **RMSE**: 7197.03  
- **MAE**: 3755.92  
- Weakest performance overall with lowest R² and highest RMSE.  
- Systematically underestimates high charges; struggles with non-linearity.

###  Decision Tree
- **R²**: 0.8981  
- **RMSE**: 4326.17  
- **MAE**: 2090.06  
- Strong accuracy, though may overfit on some predictions.

###  Random Forest
- **R²**: 0.8972  
- **RMSE**: 4345.64  
- **MAE**: 2051.15  
- Generalizes better than a single tree, with close tracking of actual values.

###  Gradient Boosting
- **R²**: 0.8970  
- **RMSE**: 4349.60  
- **MAE**: 2053.86  
- Smooth and consistent predictions, slightly below top performers.

###  Bagging
- **R²**: 0.8974  
- **RMSE**: 4342.37  
- **MAE**: 2072.77  
- Strong results, slightly more stable than a single Decision Tree.

###  Stacking
- **R²**: 0.9030 *(Best)*  
- **RMSE**: 4221.49 *(Best)*  
- **MAE**: 1982.50  
- Best model overall with highest R² and lowest RMSE.  
- Excellent accuracy and generalization.

###  Voting
- **R²**: 0.9003  
- **RMSE**: 4279.51  
- **MAE**: 1987.56  
- Nearly matches Stacking; consistent and reliable predictions.

---

###  **Best Model: Stacking**
- Highest R²: **0.9030**
- Lowest RMSE: **4221.49**
- Lowest MAE: **1982.50**


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x="smoker", y="charges", data=df)
plt.title("Insurance Charges by Smoker Status")
plt.xlabel("Smoker")
plt.ylabel("Charges")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(
    data=df,
    x="charges",
    hue="smoker",
    kde=True,
    element="step",
    stat="density",
    common_norm=False,
)
plt.title("Charges by Smoker Status")
plt.xlabel("Charges")
plt.ylabel("Density")
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
df_corr = pd.get_dummies(
    df, columns=["sex", "smoker", "region"], drop_first=True, dtype=int
)

sns.heatmap(df_corr.corr(), annot=True, cmap="coolwarm")
plt.show()

### *Smoking highly correlated to charges*

# Summary
- **Stacking** and **Gradient Boosting** show the best performance for predicting charges.
- **Smoking** appears to contribute significantly to charges and we see a high correlation of 0.79 approaching 1.
- *Age* contributes to charges. This makes sense as an individual's health deteriorates over time.
- To a lesser extent, *BMI* contributes to charges.

If you smoke, stop as soon as you can and if you're BMI is high, take steps to lower it -- because you can't control age.